In [10]:
import os
import pickle
import numpy as np
import pandas as pd
from sklearn.neighbors import KernelDensity
from scipy.signal import argrelextrema

In [11]:
DATA_DIR_PATH = "datasets/listeria"
META_DIR_PATH = "datasets"
PEAK_DIR_PATH = "datasets/extracted_peaks"

In [12]:
files =  os.listdir(DATA_DIR_PATH)
files = [fn  for fn in files if fn.endswith(".txt")]
files = sorted(files)
for file in files[:5] :
    print(file, file[:-4])

L001_0_F3_1.txt L001_0_F3_1
L001_0_F4_1.txt L001_0_F4_1
L010_0_G10_1.txt L010_0_G10_1
L010_0_G9_1.txt L010_0_G9_1
L011_0_G11_1.txt L011_0_G11_1


In [13]:
pk_files =  os.listdir(PEAK_DIR_PATH)
pk_files = [fn  for fn in pk_files if fn.endswith(".pkl")]
pk_files = sorted(pk_files)
for file in pk_files[:5] :
    print(file, file[:-10])

L100_0_G7_1_peaks.pkl L100_0_G7_1
L100_0_G8_1_peaks.pkl L100_0_G8_1
L101_0_A1_1_peaks.pkl L101_0_A1_1
L101_0_A2_1_peaks.pkl L101_0_A2_1
L102_0_A3_1_peaks.pkl L102_0_A3_1


In [14]:
extracted = []
for file in files:
    df = None
    df = pd.read_table(f"{DATA_DIR_PATH}/{file}",sep=" ", header=None,names=['m/z', 'intensity']) 
    x, y = df['m/z'].to_numpy(), df['intensity'].to_numpy()
    pk_file = f"{file[:-4]}_peaks.pkl"
    if pk_file not in pk_files:
        continue
    with open(f'{PEAK_DIR_PATH}/{pk_file}', 'rb') as peak_file:
        peaks = pickle.load(peak_file)
        # print(f'{file}, peaks: {len(peaks)}')
        extracted +=  [(x[peaks], y[peaks]) ] 

print(extracted[0][0][:5])
print(extracted[1][0][:5])

[1001.083 1007.932 1023.81  1044.463 1054.965]
[1001.273 1008.122 1024.29  1045.046 1055.16 ]


In [15]:
X = np.concatenate([x[0] for x in extracted])
# X = np.round(X, decimals=2)
X = np.round(X) # round to integer
X = np.sort(X)
X = np.unique(X)
X[-10:]

array([20063., 20065., 20066., 20067., 20068., 20069., 20070., 20071.,
       20072., 20074.])

In [16]:
def hdbscan_align(X, kernel_="gaussian", bandwidth_=100, n_samples_=4096, max_width_=3):
    """
    kde_align2(): alignment of values in an ordered list, 1-d clustering 
    Params:
      X: list[float], sorted list of m/z values to be aligned
      kernel_: str, kernel to use
      bandwidth_: float, bandwidth of the kernel, the larger the coarser / smoother  
      n_samples_: number of samples to generate
      max_width_: maximal tolerable width/diameter of a cluster
    returns: list[tuple(float,float, int)], list of tuples (min, max, n_members)
    """
    minval, maxval = X.min(), X.max()  
    kde = KernelDensity(kernel= kernel_, bandwidth=bandwidth_).fit(X.reshape(-1,1))
    s = np.linspace(minval, maxval, n_samples_)
    e = kde.score_samples(s.reshape(-1,1))
    mi = argrelextrema(e, np.less)[0]
    b = np.append(s[mi], maxval)
    groups = []
    minval_ = minval
    for v in b: 
        X_ = X[(X >= minval_) * (X < v)]
        if len(X_) > 0:
            if X_.max() - X_.min() <= max_width_:
                groups.append((X_.min(),X_.max(), len(X_)))
            else:
                groups += kde_align2(X_, bandwidth_=bandwidth_*0.618, max_width_=max_width_)
                
            minval_ = v
            
    X_ = X[(X >= minval_)]
    if len(X_) > 0:
        groups.append((X_.min(),X_.max(), len(X_))) 
        
    return groups


In [24]:
segments = kde_align2(X, bandwidth_=32, max_width_=3)
len(segments)

3834

In [25]:
from pprint import pprint 
pprint(segments[:20])
print("*" * 20)
pprint(segments[-20:])

[(1001.0, 1002.0, 2),
 (1008.0, 1008.0, 1),
 (1009.0, 1009.0, 1),
 (1017.0, 1018.0, 2),
 (1023.0, 1023.0, 1),
 (1024.0, 1024.0, 1),
 (1025.0, 1025.0, 1),
 (1033.0, 1033.0, 1),
 (1044.0, 1047.0, 4),
 (1049.0, 1049.0, 1),
 (1054.0, 1054.0, 1),
 (1055.0, 1055.0, 1),
 (1056.0, 1056.0, 1),
 (1063.0, 1066.0, 4),
 (1074.0, 1077.0, 4),
 (1078.0, 1078.0, 1),
 (1086.0, 1089.0, 4),
 (1093.0, 1093.0, 1),
 (1096.0, 1096.0, 1),
 (1097.0, 1097.0, 1)]
********************
[(20035.0, 20035.0, 1),
 (20039.0, 20039.0, 1),
 (20041.0, 20041.0, 1),
 (20043.0, 20043.0, 1),
 (20044.0, 20044.0, 1),
 (20045.0, 20045.0, 1),
 (20048.0, 20048.0, 1),
 (20051.0, 20051.0, 1),
 (20055.0, 20057.0, 2),
 (20059.0, 20059.0, 1),
 (20062.0, 20063.0, 2),
 (20065.0, 20065.0, 1),
 (20066.0, 20066.0, 1),
 (20067.0, 20067.0, 1),
 (20068.0, 20068.0, 1),
 (20069.0, 20069.0, 1),
 (20070.0, 20070.0, 1),
 (20071.0, 20071.0, 1),
 (20072.0, 20072.0, 1),
 (20074.0, 20074.0, 1)]
